In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
import numpy as np
from ipymcdm import MCDMWidget
from progressivis import (
    CSVLoader,
    Histogram2D,
    Quantiles,
    Module,
    Sink
)

In [3]:
import ipywidgets as ipw
import warnings
warnings.filterwarnings("ignore")
LARGE_TAXI_FILE = "https://www.aviz.fr/nyc-taxi/yellow_tripdata_2015-01.csv.bz2"
RESOLUTION=512

def display_histo2d(histo2d, width:int = 512, height:int = 512) -> None:
    img = MCDMWidget()
    progress = ipw.IntProgress(value=0, min=0, max=1000,
                               description='0/0',
                               orientation='horizontal')
    box = ipw.VBox([progress, img])

    display(box)  # type: ignore
    async def _after_run(m: Module, run_number: int) -> None:
        assert isinstance(m, Histogram2D)
        array = m._histo  # get the image from the heatmap
        if array is not None:
            img.array = np.log1p(np.flip(array, axis=0))  # Replace the displayed image with the new one
            # img.maxval = array.max()
        prog = m.get_progress()
        if prog is not None:
            value = prog[0]
            max = prog[1]
            progress.value = value
            progress.max = max
            if max != 0:
                percent = value * 100 / max
                progress.description = f"{int(percent)}%"

        # TODO: add a on_ending(remove _after_run)
    histo2d.on_after_run(_after_run)  # Install the callback
    return img


In [4]:

# Create a csv loader filtering out data outside NYC
csv = CSVLoader(LARGE_TAXI_FILE, index_col=False, usecols=['pickup_longitude', 'pickup_latitude'])  # , filter_=filter_)

# Create a Quantile module to get rid of the 3% outliers both sides
quantiles = Quantiles()
quantiles.input.table = csv.output.result
# Create a module to compute the 2D histogram of the two columns specified
# with the given resolution
histogram2d = Histogram2D('pickup_longitude', 'pickup_latitude', xbins=RESOLUTION, ybins=RESOLUTION)
# Connect the module to the csv results and the min,max bounds to rescale
histogram2d.input.table = csv.output.result
histogram2d.input.min = quantiles.output.result[0.03]
histogram2d.input.max = quantiles.output.result[0.97]
img = display_histo2d(histogram2d)
sink = Sink()
sink.input.inp = histogram2d.output.result

Unexpected slot hint 0.03 for Slot(quantiles_1[result]->histogram2_d_1[min])
Unexpected slot hint 0.97 for Slot(quantiles_1[result]->histogram2_d_1[max])


In [5]:
csv.scheduler().commit()
csv.scheduler().task_start()

# Scheduler added module(s): ['csv_loader_1', 'histogram2_d_1', 'quantiles_1', 'sink_1']


<Task pending name='Task-5' coro=<Scheduler.start() running at /home/fekete/src/progressivis/progressivis/core/scheduler.py:273>>

Starting scheduler


In [6]:
csv.scheduler()

Id,Class,State,Last Update,Order
csv_loader_1,csv_loader,state_ready,0,0
quantiles_1,quantiles,state_ready,0,1
histogram2_d_1,histogram2_d,state_ready,0,2
sink_1,sink,state_ready,0,3


In [9]:
img.maxval = 2
# csv.scheduler().task_stop()